In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '%.4f' % x)


In [2]:
print("import data")
ordert = pd.read_csv('order_products__train.csv')
orderp = pd.read_csv('order_products__prior.csv')

orders = pd.read_csv('orders.csv')

products = pd.read_csv('products.csv')
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')


import data


In [3]:
print("product info")
m1 = pd.merge(products, aisles, how='left', on=['aisle_id'])
products_new = pd.merge(m1, departments, how='left', on=['department_id'])

del products
del aisles
del departments
print("Size of product :", products_new.shape[0])
products_new.head()


product info
Size of product : 49688


,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,2,All-Seasons Salt,104,13,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry


In [4]:
print("merge orders and order_products__prior") 
orders = orders.fillna(value=0)
orders_products = pd.merge(orderp, orders, how='left', on=['order_id'])

del orderp
orders_products.head(10)


merge orders and order_products__prior


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0000
1,2,28985,2,1,202279,prior,3,5,9,8.0000
2,2,9327,3,0,202279,prior,3,5,9,8.0000
3,2,45918,4,1,202279,prior,3,5,9,8.0000
4,2,30035,5,0,202279,prior,3,5,9,8.0000
5,2,17794,6,1,202279,prior,3,5,9,8.0000
6,2,40141,7,1,202279,prior,3,5,9,8.0000
7,2,1819,8,1,202279,prior,3,5,9,8.0000
8,2,43668,9,0,202279,prior,3,5,9,8.0000
9,3,33754,1,1,205970,prior,16,5,17,12.0000


# Feature Extraction

### Products

In [5]:
#1. 商品第一次被購買的數量
#2. ？？？？？
#3. 商品第一次被購買的比例
#4. products_distinct_user

print("products feature info")

f = {'reordered':['count', 'sum']}
g1 = orders_products.groupby(["product_id"]).aggregate(f).reset_index()
g1.columns = ["product_id",
              "prod_orders", "prod_reorders"]

g1["prod_first_orders"] = g1["prod_orders"] - g1["prod_reorders"]
g1["prod_reorder_times"] = 1 + g1["prod_reorders"] / g1["prod_first_orders"]
g1["prod_reorder_ratio"] = g1["prod_reorders"] / g1["prod_orders"]
g1 = g1.drop(["prod_reorders", "prod_first_orders"], axis = 1)

g2_1 = orders_products.groupby(["user_id", "product_id"])["reordered"].aggregate({"count_reorder": "count"}).reset_index()
g2 = g2_1.groupby(["product_id"])["user_id"].aggregate({"products_distinct_user": "count"}).reset_index()

prd = pd.merge(g1, g2, how='left', on=['product_id'])

del g1
del g2_1
del g2
print("nb of products features :", prd.shape[1])
prd.head()

products feature info
nb of products features : 5


,product_id,prod_orders,prod_reorder_times,prod_reorder_ratio,products_distinct_user
0,1,1852,2.5866,0.6134,716
1,2,90,1.1538,0.1333,78
2,3,277,3.7432,0.7329,74
3,4,329,1.8077,0.4468,182
4,5,15,2.5000,0.6000,6


1. groupby products in orders_products
2. 商品的變數新增：購買次數，？？？，重複被購買比例

### Users

In [6]:
# 總共消費天數 of prior
# 平均間隔時間 of prior
# 總共購買次數 of prior

print("users feature info")
orders_prior = orders[orders.eval_set == "prior"]
f = {'days_since_prior_order':['sum', 'mean'],
     'order_number':['max']}
us0 = orders_prior.groupby(["user_id"]).aggregate(f).reset_index()
us0.columns = ["user_id",
              "user_period", "user_mean_days_since_prior",
              "user_orders"]
us0["up_nb_order"] = us0.user_orders + 1 #user_total_order
us0.head()


users feature info


,user_id,user_period,user_mean_days_since_prior,user_orders,up_nb_order
0,1,176.0000,17.6000,10,11
1,2,198.0000,14.1429,14,15
2,3,133.0000,11.0833,12,13
3,4,55.0000,11.0000,5,6
4,5,40.0000,10.0000,4,5


In [7]:
# 總共購買商品
# 重複購買比例
# 購買商品種類

f = {'product_id':['count']}
g1 = orders_products.groupby(["user_id"]).aggregate(f).reset_index()
g1.columns = ["user_id", "user_total_product"]

g2 = orders_products[orders_products.order_number > 1].groupby("user_id")["reordered"].aggregate({"count_reorder": "sum",
                                                                                                  "count_order": "count"}).reset_index()
g2["user_reorder_ratio"] = g2["count_reorder"] / g2["count_order"]

g3_1 = orders_products.groupby(["user_id", "product_id"])["reordered"].aggregate({"count_reorder": "count"}).reset_index()
g3 = g3_1.groupby(["user_id"])["product_id"].aggregate({"user_distinct_products": "count"}).reset_index()

us1 = pd.concat([g1, g2[["user_reorder_ratio"]], g3[["user_distinct_products"]]], axis = 1)

del g1
del g2
del g3_1
del g3
us1.head()


,user_id,user_total_product,user_reorder_ratio,user_distinct_products
0,1,59,0.7593,18
1,2,195,0.5110,102
2,3,88,0.7051,33
3,4,18,0.0714,17
4,5,37,0.5385,23


1. 有些顧客重複購買率超高，有些根本超低
2. 對於購買數量超多的顧客，reordered ratio都不低
3. 幾乎都是購買數量少的顧客，不重複購買。

In [8]:
# 取出train and test，以及當時購買資訊

orders_new = orders[["user_id", "order_id", "eval_set", "order_dow", "order_hour_of_day"]]
us2 = orders_new[(orders_new.eval_set != "prior")]
us2.head()


,user_id,order_id,eval_set,order_dow,order_hour_of_day
10,1,1187899,train,4,8
25,2,1492625,train,1,11
38,3,2774568,test,5,15
44,4,329954,test,3,12
49,5,2196797,train,0,11


In [9]:
# 合併三個有關users的資料
#顧客平均購買的商品數量

m1 = pd.merge(us0, us1, how='left', on=['user_id'])
users = pd.merge(m1, us2, how='left', on=['user_id'])
users["user_average_basket"] = users["user_total_product"] / users["user_orders"]

del us0
del us1
del us2
del m1
print("nb of users features :", users.shape[1])
users.head()


nb of users features : 13


,user_id,user_period,user_mean_days_since_prior,user_orders,up_nb_order,user_total_product,user_reorder_ratio,user_distinct_products,order_id,eval_set,order_dow,order_hour_of_day,user_average_basket
0,1,176.0000,17.6000,10,11,59,0.7593,18,1187899,train,4,8,5.9000
1,2,198.0000,14.1429,14,15,195,0.5110,102,1492625,train,1,11,13.9286
2,3,133.0000,11.0833,12,13,88,0.7051,33,2774568,test,5,15,7.3333
3,4,55.0000,11.0000,5,6,18,0.0714,17,329954,test,3,12,3.6000
4,5,40.0000,10.0000,4,5,37,0.5385,23,2196797,train,0,11,9.2500


In [10]:
#users.to_csv("users.csv", index = False)
#prd.to_csv("prd.csv", index = False)


In [11]:
users = pd.read_csv("users.csv")
prd = pd.read_csv("prd.csv")

print("size of prd features :", prd.shape[1])
print("size of users features :", users.shape[1])

f = lambda x: np.array_split(list(x), 1)[0]
g = orders.groupby('user_id')["days_since_prior_order"].apply(f).reset_index() #user_list_days_since_prior_order
users = pd.merge(users, g, how='left', on=['user_id'])


size of prd features : 5
size of users features : 13


### Users and Products

In [13]:
# groupby users and products in orders_products
# 商品顧客組合變數新增：出現幾次，第一次出現時間，最後一次出現時間，出現在購物籃的順序平均

f = {'order_number':['count', 'min', 'max'],
     'add_to_cart_order':['mean'],
     'order_dow':['mean', 'std'],
     'days_since_prior_order':['mean', 'std'],
     'order_hour_of_day':['mean', 'std']}
usersXprod = orders_products.groupby(["user_id", "product_id"]).aggregate(f).reset_index()
usersXprod.columns = ["user_id", "product_id",
                      "up_orders", "up_first_order", "up_last_order",
                      "up_average_cart_position", 
                      'up_mean_order_dow', 'up_var_order_dow',
                      'up_mean_days_since_prior_order', 'up_var_days_since_prior_order',
                      'up_mean_order_hour_of_day', 'up_var_order_hour_of_day']

usersXprod = usersXprod.fillna(value=0)
usersXprod['user_product'] = usersXprod.product_id + usersXprod.user_id * 100000

print("nb of usersXproducts features :", usersXprod.shape[1])
usersXprod.head()


nb of usersXproducts features : 13


,user_id,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position,up_mean_order_dow,up_var_order_dow,up_mean_days_since_prior_order,up_var_days_since_prior_order,up_mean_order_hour_of_day,up_var_order_hour_of_day,user_product
0,1,196,10,1,10,1.4000,2.5000,1.2693,17.6000,10.8033,10.3000,3.5917,100196
1,1,10258,9,2,10,3.3333,2.5556,1.3333,19.5556,9.3956,10.5556,3.7118,110258
2,1,10326,1,5,5,5.0000,4.0000,0.0000,28.0000,0.0000,15.0000,0.0000,110326
3,1,12427,10,1,10,3.3000,2.5000,1.2693,17.6000,10.8033,10.3000,3.5917,112427
4,1,13032,3,2,10,6.3333,2.6667,1.5275,21.6667,7.6376,8.0000,1.0000,113032


### 合併三個資料(Users, Products, Users and Products)

In [14]:
print("merge prod, user and usersXprod feature on usersXprod")
usersXprod = pd.merge(usersXprod, prd, how='left', on=["product_id"])
usersXprod = pd.merge(usersXprod, users, how='left', on=["user_id"])

### 顧客購買此商品的比例
### 顧客上次購買之後幾次沒買
### 這個組合出現的次數 / (顧客購買次數-這個組合第一次出現的次數+1)
usersXprod["up_order_rate"] = usersXprod["up_orders"] / usersXprod["user_orders"]
usersXprod["up_orders_since_last_order"] = usersXprod["user_orders"] - usersXprod["up_last_order"]
usersXprod["up_order_rate_since_first_order"] = usersXprod["up_orders"] / (usersXprod["user_orders"] - usersXprod["up_first_order"] + 1)

del prd
del users
usersXprod.head()


merge prod, user and usersXprod feature on usersXprod


,user_id,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position,up_mean_order_dow,up_var_order_dow,up_mean_days_since_prior_order,up_var_days_since_prior_order,...,user_distinct_products,order_id,eval_set,order_dow,order_hour_of_day,user_average_basket,days_since_prior_order,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order
0,1,196,10,1,10,1.4000,2.5000,1.2693,17.6000,10.8033,...,18,1187899,train,4,8,5.9000,"[0.0, 15.0, 21.0, 29.0, 28.0, 19.0, 20.0, 14.0...",1.0000,0,1.0000
1,1,10258,9,2,10,3.3333,2.5556,1.3333,19.5556,9.3956,...,18,1187899,train,4,8,5.9000,"[0.0, 15.0, 21.0, 29.0, 28.0, 19.0, 20.0, 14.0...",0.9000,0,1.0000
2,1,10326,1,5,5,5.0000,4.0000,0.0000,28.0000,0.0000,...,18,1187899,train,4,8,5.9000,"[0.0, 15.0, 21.0, 29.0, 28.0, 19.0, 20.0, 14.0...",0.1000,5,0.1667
3,1,12427,10,1,10,3.3000,2.5000,1.2693,17.6000,10.8033,...,18,1187899,train,4,8,5.9000,"[0.0, 15.0, 21.0, 29.0, 28.0, 19.0, 20.0, 14.0...",1.0000,0,1.0000
4,1,13032,3,2,10,6.3333,2.6667,1.5275,21.6667,7.6376,...,18,1187899,train,4,8,5.9000,"[0.0, 15.0, 21.0, 29.0, 28.0, 19.0, 20.0, 14.0...",0.3000,0,0.3333


### 特別的 days_since_prior_order

In [15]:
prior_order = usersXprod[["up_last_order","up_nb_order","days_since_prior_order","user_product"]]

d= dict()
i = 0
for i, row in enumerate(prior_order.itertuples(), 1):
    i+=1
    if i%2000000 == 0: print ('order row', i)    
    z = row.user_product
    last = int(row.up_last_order)
    now = int(row.up_nb_order)
    prior = sum(row.days_since_prior_order[last: now])
    d[z] = prior

p = pd.DataFrame.from_dict(d, orient = 'index')
del d
del prior_order


order row 2000000
order row 4000000
order row 6000000
order row 8000000
order row 10000000
order row 12000000


In [17]:
p["user_product"] = p.index
p.columns = ["up_days_since_prior_order", "user_product"]
usersXprod = pd.merge(usersXprod, p, how='left', on=["user_product"])
usersXprod = usersXprod.drop(["up_nb_order","days_since_prior_order","user_product"], axis=1)

del p
usersXprod.head()


,user_id,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position,up_mean_order_dow,up_var_order_dow,up_mean_days_since_prior_order,up_var_days_since_prior_order,...,user_distinct_products,order_id,eval_set,order_dow,order_hour_of_day,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,up_days_since_prior_order
0,1,196,10,1,10,1.4000,2.5000,1.2693,17.6000,10.8033,...,18,1187899,train,4,8,5.9000,1.0000,0,1.0000,14.0000
1,1,10258,9,2,10,3.3333,2.5556,1.3333,19.5556,9.3956,...,18,1187899,train,4,8,5.9000,0.9000,0,1.0000,14.0000
2,1,10326,1,5,5,5.0000,4.0000,0.0000,28.0000,0.0000,...,18,1187899,train,4,8,5.9000,0.1000,5,0.1667,97.0000
3,1,12427,10,1,10,3.3000,2.5000,1.2693,17.6000,10.8033,...,18,1187899,train,4,8,5.9000,1.0000,0,1.0000,14.0000
4,1,13032,3,2,10,6.3333,2.6667,1.5275,21.6667,7.6376,...,18,1187899,train,4,8,5.9000,0.3000,0,0.3333,14.0000


### 加入reorder in train

In [18]:
### ordert and orders merge
ordert = pd.merge(ordert, orders, how='left', on=['order_id'])

del orders
ordert.head()


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,112108,train,4,4,10,9.0000
1,1,11109,2,1,112108,train,4,4,10,9.0000
2,1,10246,3,0,112108,train,4,4,10,9.0000
3,1,49683,4,0,112108,train,4,4,10,9.0000
4,1,43633,5,1,112108,train,4,4,10,9.0000


> reordered=0的商品為原本prior沒見過的，所以在下面結合的時候，也不會有0出現

In [19]:
ordert = ordert[["user_id", "product_id", "reordered"]]
usersXprod = pd.merge(usersXprod, ordert, how='left', on=["user_id", "product_id"])

del ordert
usersXprod.head()


,user_id,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position,up_mean_order_dow,up_var_order_dow,up_mean_days_since_prior_order,up_var_days_since_prior_order,...,order_id,eval_set,order_dow,order_hour_of_day,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,up_days_since_prior_order,reordered
0,1,196,10,1,10,1.4000,2.5000,1.2693,17.6000,10.8033,...,1187899,train,4,8,5.9000,1.0000,0,1.0000,14.0000,1.0000
1,1,10258,9,2,10,3.3333,2.5556,1.3333,19.5556,9.3956,...,1187899,train,4,8,5.9000,0.9000,0,1.0000,14.0000,1.0000
2,1,10326,1,5,5,5.0000,4.0000,0.0000,28.0000,0.0000,...,1187899,train,4,8,5.9000,0.1000,5,0.1667,97.0000,nan
3,1,12427,10,1,10,3.3000,2.5000,1.2693,17.6000,10.8033,...,1187899,train,4,8,5.9000,1.0000,0,1.0000,14.0000,nan
4,1,13032,3,2,10,6.3333,2.6667,1.5275,21.6667,7.6376,...,1187899,train,4,8,5.9000,0.3000,0,0.3333,14.0000,1.0000


In [20]:
usersXprod.reordered = usersXprod.reordered.replace(to_replace = "nan", value = 0)
usersXprod.head()


,user_id,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position,up_mean_order_dow,up_var_order_dow,up_mean_days_since_prior_order,up_var_days_since_prior_order,...,order_id,eval_set,order_dow,order_hour_of_day,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,up_days_since_prior_order,reordered
0,1,196,10,1,10,1.4000,2.5000,1.2693,17.6000,10.8033,...,1187899,train,4,8,5.9000,1.0000,0,1.0000,14.0000,1.0000
1,1,10258,9,2,10,3.3333,2.5556,1.3333,19.5556,9.3956,...,1187899,train,4,8,5.9000,0.9000,0,1.0000,14.0000,1.0000
2,1,10326,1,5,5,5.0000,4.0000,0.0000,28.0000,0.0000,...,1187899,train,4,8,5.9000,0.1000,5,0.1667,97.0000,0.0000
3,1,12427,10,1,10,3.3000,2.5000,1.2693,17.6000,10.8033,...,1187899,train,4,8,5.9000,1.0000,0,1.0000,14.0000,0.0000
4,1,13032,3,2,10,6.3333,2.6667,1.5275,21.6667,7.6376,...,1187899,train,4,8,5.9000,0.3000,0,0.3333,14.0000,1.0000


1. 將ordert, orders合併
2. 將order_train的reorder取出當作y
3. order_train裡面reordered=1的商品，是prior有出現的組合
4. 在train第一次購買的商品可能就無法考慮
5. 而prior裡面有些組合train沒有出現，所以出現nan我們改填為0

## 顧客商品
up_orders：此組合出現幾次  
up_first_order：此組合第一次出現時間  
up_last_order：此組合最後一次出現時間  
up_average_cart_position：此組合出現在購物籃的順序平均  
up_order_rate：顧客購買此商品的比例  
up_orders_since_last_order：顧客上次購買之後幾次沒買  
up_order_rate_since_first_order：這個組合出現的次數 / (顧客購買次數-這個組合第一次出現的次數+1)

## 商品
prod_orders：商品出現次數  
prod_reorder_times：？？？  
prod_reorder_ratio：商品被重複購買比例  

## 顧客
user_total_product：總購買商品數量  
user_reorder_ratio：總購買數量中重複購買的比例  
user_distinct_products：購買不同商品種類  
user_average_basket：平均每次購買數量  
##### prior
user_orders：總購買次數  
user_period：總購買間隔時間  
user_mean_days_since_prior：平均購買間隔時間  
##### Train
days_since_prior_order：距離上次購買的時間  